In [1]:
import os

# imdb title conventions
https://contribute.imdb.com/updates/guide/title_formats

- Title (year)
- Drop it for: in quotes, (TV / V / VG )
- Drop it for anything with {{


In [2]:
def processTitle ( title ):
    valid = True
    if ( title.startswith('"')):
        valid = False
    elif '(V)' in title or '(TV)' in title or '(VG)' in title:
        valid = False
    elif ('}' in title ):
        valid = False
    else:
        titParts = title.split(')')
        title = titParts[0] + ')'
        
    return (valid, title)

print(processTitle( "\Dante\'s Domicile\" (2013) {The Hearse: Part 1 (#2.5)}  (as Dante Darrk)"))
print(processTitle("The 1996 Fox Rock 'N Roll Skating Championships (1996) (TV)"))
print(processTitle("At Night (2008)  (co-director)"))
print(processTitle("Freddie Goes to Washington (1993) {{SUSPENDED}}"))

(False, '\\Dante\'s Domicile" (2013) {The Hearse: Part 1 (#2.5)}  (as Dante Darrk)')
(False, "The 1996 Fox Rock 'N Roll Skating Championships (1996) (TV)")
(True, 'At Night (2008)')
(False, 'Freddie Goes to Washington (1993) {{SUSPENDED}}')


This is the directory where "directors.list" and "ratings.list" can be found. I downloaded them from ftp://ftp.funet.fi/pub/mirrors/ftp.imdb.com/pub/

See description at http://www.imdb.com/interfaces

In [3]:
fileDir = r"F:\danyelf\OneDrive - Microsoft\Projects\2016 Making Sense of Data\datasets\\"

# Directors 

What we're going to do it wrap it all in a big ol' for loop.

1.	Skip to line 236
2.	If line starts with character, copy name to buffer
3.	If line starts with three tabs, write intead (name, title)
4.	Write line

In [4]:
path = fileDir + "directors.list"
outputpath = fileDir + "cleandirectors.list"

directorDictionary = {}

with open( path, "r") as fo:
    with open(outputpath, "w", buffering = 1) as out:
        print("Name of file: ", fo.name)
        lineNo = 0
        outline = 0
        director = "zzzz"
        for line in fo:
            lineNo += 1
            if (lineNo < 236): 
                continue
            elif ("----------------------" in line):
                break
            elif ( len( line ) < 2): # The gap between movies
                continue
            elif (line.startswith ('\t')):
                (valid, title) = processTitle( line.strip() );
            else: 
                parts = line.split('\t')
                director = parts[0]
                (valid, title) = processTitle( parts[-1].rstrip());

            if( valid ):
                directorDictionary [ title ] = director
 #               print(director, "     ", title)        
                out.write( director + "\t" + title + "\n")
                outline += 1
                if( outline % 1000 == 0):
                    print (outline, director, "    ", title)
#            if ( outline > 100000):
#                break

Name of file:  F:\danyelf\OneDrive - Microsoft\Projects\2016 Making Sense of Data\datasets\\directors.list
1000 Abbott, Lee (I)      Rain (2005)
2000 Abidov, Igor      Svet nizhnikh kvartalov (2018)
3000 Abuel, Miguel      Drunk People (2014)
4000 Adachi, Satoshi (III)      Sukûru obu nâshingu (2016)
5000 Adel, Soufiane      Go Forth (2014)
6000 Affleck, Ben      I Killed My Lesbian Wife, Hung Her on a Meathook, and Now I Have a Three Picture Deal at Disney (1993)
7000 Aguerre, Roberto      Ojos de amatista (1996)
8000 Ahmad, Qalandar      Bodi Shah (1959)
9000 Aix, Alain d'      Mercenaires en quête d'auteurs (1983)
10000 Akkadej, Kom      Killer Elephants (1976)
11000 Aladag, Baris      6 Tage - 6 Genres (2006)
12000 Albert, Rafa      Sara No És Necessària (1998)
13000 Alegre, Alejandro G.      Los infectados (2011)
14000 Alexius, Tirf      Cerrrrmack (2010)
15000 Alkire, Spike      Mom's Ashes (2013)
16000 Alli, Antero      To Dream of Falling Upwards (2011)
17000 Alonso, Carlos Alb

In [5]:
print( len(directorDictionary))
print ( directorDictionary['Star Wars (1977)'] )
print( directorDictionary['The Usual Suspects (1995)'])

891210
Lucas, George (I)
Singer, Bryan


# Ratings

The columns are
`New  Distribution  Votes  Rank  Title`

Each line looks like
`      2000.0.004      56   6.4  "#BlackLove" (2015)`

We only care about VOTES, RANK, and TITLE
- Votes: column 17-25
- Rank:  column 27-31
- Title: starts at column 33 to [-1] (then there's a newline)


1. Skip to line 297 (we dont need the top 250 or the bottom 50)
2. Stop at the line that starts with "-----------"

In [6]:
path = fileDir + "ratings.list"
outputpath = fileDir + "cleanratings.list"
noDirPath = fileDir + "nodirector.list"


with open( path, "r") as fo:
    with open(outputpath, "w", buffering = 1) as out:
        with open(noDirPath, "w") as outerr:           
            print("Name of file: ", fo.name)
            out.write("raters\tscore\ttitle\tdirector\n")
            lineNo = 0
            outline = 0
            director = "zzzz"
            for line in fo:
                lineNo += 1
                if (lineNo < 297): 
                    continue
                elif ( len(line) < 2):
                    continue
                elif ("----------------------" in line):
                    break

                numRatings = int(line[17:25])
                avgRank = float( line[27:31])
                (valid, title) = processTitle(line[32:-1].lstrip())

                if( valid ):

                    ln =  str(numRatings) + "\t" + str(avgRank) + "\t" + title

                    if( title in directorDictionary):
                        director = directorDictionary[ title ]
                        ln += "\t" + director
                        out.write( ln  + "\n")
                    else:
                        outerr.write( ln  + "\n")

                    outline += 1
                    if( outline % 1000 == 0):
                        print( ln )


Name of file:  F:\danyelf\OneDrive - Microsoft\Projects\2016 Making Sense of Data\datasets\\ratings.list
217	6.6	125 rue Montmartre (1959)	Grangier, Gilles
5	5.4	3 (1995)	McKinnon, Morag
754	5.2	666 - Traue keinem, mit dem Du schläfst! (2002)	Matsutani, Rainer
77741	6.8	A Christmas Carol (2009)	Zemeckis, Robert
6	4.5	A Handful of Dust (1974)	Kuyululu, Ayten
6	7.8	A Nona Vítima (2012)	Zon, Diego
7	6.9	A Tortured Mind (2012)	Wade, Dave
7	6.4	Aadhi Mimansa (1991)	Bir, Apurba Kishore
20	6.2	Accidentally on Purpose (2005)	Shires, Claude (I)
246	3.9	Adventures of a Plumber's Mate (1978)	Long, Stanley A.
26	6.2	Ahankaar (1995)	Samanta, Ashim S.
62	7.0	Al-saqqa mat (1981)	Abouseif, Salah (I)
64	6.6	All Ages Night (2009)	Stein, Nancy Montuori
17	7.1	Alt er relativt (2008)	Krogh, Mikala
6	9.3	American Terrorist (2011)	Sherman, Ben (IV)
16	5.1	Anak setan (2009)	Geaffary, Allo
5	8.6	Animal Attraction (2012)	Norton, Bryan (II)
18	4.9	Anyab (1981)	Shebl, Mohammed
93	4.7	Area X (2007)	Shinners, Denni

In [7]:
s = '      2000.0.004      56   6.4  "#BlackLove" (2015)'
print( int(s[17:25]))
print( float(s[27:31]))
print( s[33:-1])

56
6.4
#BlackLove" (2015


In [14]:
import sys
print( sys.stdout.encoding )

UTF-8
